read in hyperparameter matrix (output from opt.py: "wname"\_optimize_result.npy)

start george.GP object with kernels and set parameter vector

In [1]:
import numpy as np
import george
from george import kernels
from scipy.optimize import minimize
from time import time
import argparse

In [ ]:
parser=argparse.ArgumentParser()
parser.add_argument("--name", help="name of pca instance",type=str)

In [ ]:
coords=np.load("./etgrid/"+args.rname+"_coords.npy")
eigenseds=np.load("./etgrid/"+args.rname+"_eigenseds.npy")
weights=np.load("./etgrid/"+args.rname+"_weights.npy")
pcamean=np.load("./etgrid/"+args.rname+"_mean.npy")
sedsflat=np.load("./etgrid/sedsflat.npy")

yerrs=[]
for i in range(16):
    yerrs.append([x*0.01 for x in weights[i]])

hyperparams=np.load(parser.parse_args().name+"_optimize_result.npy")

In [2]:
kernel = 16*kernels.ExpSquaredKernel(15**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=5)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=14) 
blankhodlr=george.GP(kernel)

In [ ]:
gp=blankhodlr
preds=[]
for i in range(len(weights)):  # same covfunc for each weight and the sample mean
    gp.set_parameter_vector(hyperparams[i])
    if args.verbose==True:
        print("weight #"+str(i))
        print(gp.get_parameter_vector())
    gp.compute(coords,yerrs[i])
    pred, pred_var = gp.predict(weights[i], coords, return_var=True)
    preds.append(pred)
reconst_SEDs=[]
for i in range(len(coords)):
    reconst=np.dot(np.array(preds)[:,i][0:15],eigenseds[0:15]) + pcamean + np.array(preds)[:,i][15]
    reconst_SEDs.append(reconst)
allsedsflat=np.ndarray.flatten(np.array(reconst_SEDs))
chisq=np.sum((sedsflat-allsedsflat)**2/0.1)
if args.verbose==True:
    print(chisq)
return chisq